In [1]:
from mxnet import gluon

net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Conv2D(channels=20,kernel_size=5,activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2,strides=2))
    net.add(gluon.nn.Conv2D(channels=50,kernel_size=3,activation='relu'))
    net.add(gluon.nn.MaxPool2D(pool_size=2,strides=2))
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(128,activation='relu'))
    net.add(gluon.nn.Dense(10))

In [2]:
import sys
sys.path.append('..')
import utils

ctx = utils.try_gpu()
net.initialize(ctx=ctx)

print('initialize weight on',ctx)

initialize weight on gpu(0)


In [3]:
from utils import load_data_fashion_mnist

batch_size = 256
train_data,test_data = load_data_fashion_mnist(batch_size)

In [8]:
from mxnet import autograd as autograd
from utils import sgd,accuracy,evaluate_accuracy
from mxnet import gluon
from mxnet import nd

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.5})

for epoch in range(5):
    trian_loss = 0.
    train_acc = 0.
    for data,label in train_data:
        label = label.as_in_context(ctx)
        with autograd.record():
            output = net(data.as_in_context(ctx))
            loss = softmax_cross_entropy(output,label)
        loss.backward()
        trainer.step(batch_size)

        trian_loss += nd.mean(loss).asscalar()
        train_acc += accuracy(output.astype('float32'),label.astype('float32'))
    test_acc = evaluate_accuracy(test_data,net,ctx)
    print("Epoch %d,Loss: %f,Train acc: %f,Test acc: %f"%(epoch,trian_loss/len(train_data),train_acc/len(train_data),
    test_acc))

Epoch 0,Loss: 0.922571,Train acc: 0.648016,Test acc: 0.020847
Epoch 1,Loss: 0.444093,Train acc: 0.832142,Test acc: 0.021550
Epoch 2,Loss: 0.368154,Train acc: 0.862240,Test acc: 0.021847
Epoch 3,Loss: 0.329126,Train acc: 0.876374,Test acc: 0.022113
Epoch 4,Loss: 0.303699,Train acc: 0.886259,Test acc: 0.022225
